In [7]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
df=pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

In [8]:
df.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
print ("Number of columns in jan 2023 data set : ", df.shape[1])

Number of columns in jan 2023 data set :  19


In [5]:
df['duration']= (df.tpep_dropoff_datetime - df.tpep_pickup_datetime)
df['duration1']=(df.duration.dt.total_seconds())/60
std_dev= df['duration1'].std()
print(df.duration)
print (df.duration1)
print(f"Standard deviation of trip durations: {std_dev:.2f} minutes")

0         0 days 00:08:26
1         0 days 00:06:19
2         0 days 00:12:45
3         0 days 00:09:37
4         0 days 00:10:50
                ...      
3066761   0 days 00:13:59
3066762   0 days 00:19:27
3066763   0 days 00:24:31
3066764   0 days 00:13:00
3066765   0 days 00:14:24
Name: duration, Length: 3066766, dtype: timedelta64[us]
0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration1, Length: 3066766, dtype: float64
Standard deviation of trip durations: 42.59 minutes


In [6]:
df_filtered = df[(df['duration1'] >= 1) & (df['duration1'] <= 60)]
# Calculate the fraction of data retained
fraction_retained = len(df_filtered) / len(df)
print(f"Fraction of data retained: {fraction_retained:.2%}")

Fraction of data retained: 98.12%


In [7]:
df_filtered.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration,duration1
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,...,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,0 days 00:08:26,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,...,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,0 days 00:06:19,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,...,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,0 days 00:12:45,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,...,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,0 days 00:09:37,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,...,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,0 days 00:10:50,10.833333


In [11]:
categorical= ['PULocationID', 'DOLocationID']
numerical=['trip_distance']
df_filtered[categorical]= df_filtered[categorical].astype(str)
dicts=df_filtered[categorical+numerical].to_dict(orient='records')

/tmp/ipykernel_61164/3308434961.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[categorical]= df_filtered[categorical].astype(str)


In [37]:
dv = DictVectorizer()
X_train= dv.fit_transform(dicts)
print ("Dimensionality is :" ,X_train.shape[1])
Y_train=df_filtered['duration1'].values

from sklearn.metrics import mean_squared_error
import numpy as np
lr=LinearRegression() 
lr.fit(X_train, Y_train)
Y_pred= lr.predict(X_train)
rmse=np.sqrt(mean_squared_error(Y_train, Y_pred))
print(f"RMSE: {rmse:.2f}")

Dimensionality is : 516
RMSE: 7.66


In [21]:
validation_df=pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')
validation_df.head()
validation_df['duration']=validation_df.tpep_dropoff_datetime- validation_df.tpep_pickup_datetime
validation_df['duration1']=(validation_df.duration.dt.total_seconds()/60)
validation_df_filtered= validation_df[(validation_df.duration1 >=1) & (validation_df.duration1 <=60)]
validation_df_filtered.head()

categorical_val= ['PULocationID', 'DOLocationID']
numerical_val= ['trip_distance']
validation_df_filtered[categorical_val]=validation_df_filtered[categorical_val].astype(str)

validation_dicts= validation_df_filtered[categorical_val+numerical_val].to_dict(orient='records')

dval=DictVectorizer()
X_Val= dval.fit_transform(validation_dicts) 
print ("Validation dime", X_Val.shape[1]) 
Y_Val=validation_df_filtered['duration1'].values

/tmp/ipykernel_69925/1274907934.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_df_filtered[categorical_val]=validation_df_filtered[categorical_val].astype(str)


Validation dime 515


In [23]:
from sklearn.metrics import mean_squared_error
import numpy as np
lr=LinearRegression()
lr.fit(X_Val, Y_Val)
Y_Val_pred=lr.predict (X_Val)
valrmse=np.sqrt(mean_squared_error(Y_Val_pred, Y_Val)) 
print (f"RMSE on validation: {valrmse: .2f}")

RMSE:  7.79
